In [1]:
#@ IMPORTING THE REQUIRED DEPENDENCIES
import cv2
import pytesseract

In [2]:
#@ READING THE IMAGE
image = cv2.imread("data/sample_mgh.jpeg")
base_image = image.copy()

In [3]:
#@ CONVERTING THE IMAGE TO GRAYSCALE
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#@ APPLY GAUSSIAN BLUR TO REDUCE NOISE
blur = cv2.GaussianBlur(gray, (7,7), 0)

#@ APPLYING THRESHOLDING TO CREATE BINARY IMAGE USING OTSU'S 
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

#@ CREATE STRUCTURING ELEMENT FOR MORPHOLOGICAL OPERATIONS
kernal = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 50))

#@ DILATE THRESHOLDED IMAGE USING STRUCTURING ELEMENT TO ENHACE WHITE REGIONS AND FILL GAPS
dilate = cv2.dilate(thresh, kernal, iterations=1)

cv2.imwrite("temp/sample_dilated.png", dilate)

True

In [4]:
#@ FIND CONTOURS IN DILATED IMAGE
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#@ EXTRACT THE CONTOURS FROM THE RETURNED VALUE 
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

#@ ORDERE THE CONTOURS FROM TOP TO BOTTOM
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[1])

In [5]:
#@ Iterateover each contour in list of contours
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)                                      # Get bounding rectangle coordinates and dimensions
    if h > 200 and w > 250:                                            # Check the height
        roi = base_image[y:y+h, x:x+w]                                 # Extract the ROI from base image
        cv2.rectangle(image, (x,y), (x+w, y+h), (36, 255, 12), 2)      # Draw green rectangle on orginal image


cv2.imwrite("temp/sample_boxes.png", image)                            # Save image with drawn bouding boxes 

True

In [7]:
#@ PERFORM OCR ON THE BASE IMAGE 
orginal_ocr = pytesseract.image_to_string(base_image)
print(orginal_ocr)

18 ALCVINI EPISTOLAE. 1.

1,

773-786 Alevinus amicum in monasterium se abditurum confirmat. De Benedicto monacho.
773—786.
Cod. H fol. 61'—62'.
Edd. F211, J. 8, — Migne col. 485.

DILECTISSIMO* AMICO TOTIUS PROSPERITATIS PRAESENTIS ET AETERNAE
BEATITUDINIS PERPETUAM SALUTEM.
Magna mihi laetitia est de bona voluntate vestra, quam audivi a fratre nostro
Benedicto! in vobis esse. Opto atque Deum deprecor, ut citius cum omni convenien-
Pont tia perficiatur. Scriptum est enim: ‘Ne tardes converti ad dominum Deum; quia
nescis, quid ventura pariat dies’, Erue te de harum carcere tribulationum, quae in
Ps. 33, 20.hoe mundo fidelium animos torquere solent®; sicut scriptum est: ‘Multae tribulationes
iustorum’; ut, quod sequitur, tibi evenire merearis: ‘Sed de his omnibus liberavit eos
cf. Luc.9,62. Dominus’. Et cave diligentissime, ne qua te, aratrum Domini tenentem, iniustitia
retro revocet. Nemo miles sarcinis alienis onustus ad bella bene procedit, nisi armis
tantummodo victricibus, vel ad d

In [9]:
#@ PERFORM OCR ON ROI IMAGE 
new_ocr = pytesseract.image_to_string(roi)
print(new_ocr)

ae

Alevinus amicum in monasterium se abditurum confirmat, De Benedicto monacho.
773—786.

Cod. H fol. 61'—62'.

Edd. F211, J. 8, — Migne col. 485.

DILECTISSIMO* AMICO TOTIUS PROSPERITATIS PRAESENTIS ET AETERNAE
BEATITUDINIS PERPETUAM SALUTEM.

Magna mihi laetitia est de bona voluntate vestra, quam audivi a fratre nostro
Benedicto! in vobis esse. Opto atque Deum deprecor, ut citius cum omni convenien-
tia perficiatur. Scriptum est enim: ‘Ne tardes converti ad dominum Deum; quia
nescis, quid ventura pariat dies’, Erue te de harum carcere tribulationum, quae in
hoe mundo fidelium animos torquere solent®; sicut scriptum est: ‘Multae tribulationes
iustorum’; ut, quod sequitur, tibi evenire merearis: ‘Sed de his omnibus liberavit eos
Dominus’. Et cave diligentissime, ne qua te, aratrum Domini tenentem, iniustitia
retro revocet. Nemo miles sarcinis alienis onustus ad bella bene procedit, nisi armis
tantummodo victricibus, vel ad defensionem sui vel ad laesionem adversarii.

Omnia quae vobis

In [13]:
#@ SEPARATING A FOOTNOTE FROM THE BODY TEXT

image = cv2.imread('data/sample_mgh.jpeg')                                               # Read the input image
im_h, im_w, im_d = image.shape                                                           # Get the dimensions of the image
base_image = image.copy()                                                                # Create a copy of the image as the base image

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)                                           # Convert the image to grayscale
blur = cv2.GaussianBlur(gray, (7, 7), 0)                                                 # Apply Gaussian blur to reduce noise
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]         # Apply thresholding to create a binary image

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 10))                             # Create a rectangular structuring element for dilation
dilate = cv2.dilate(thresh, kernel, iterations=1)                                        # Dilate the binary image to enhance the regions

#@ Find contours in the dilated image
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[1])

#@ Iterate over the contours
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    
    if h < 20 and w > 250:                                                              # Check if contour represents a footnote
        roi = base_image[0:y+h, 0:x+im_w]                                               # Extract region of interest (ROI) from the base image
        cv2.rectangle(image, (x, y), (x + w, y + h), (36, 255, 12), 2)                  # Draw rectangle around ROI to visualize the separation

# Save the ROI as the output image
cv2.imwrite("temp/output.png", roi)

True